In [11]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET

In [52]:
# Construct empty DataFrame
df_premises = pd.DataFrame(columns=[
    'file_idx', 'file_path', 'split', 'premise', 'type'
])

def parse_xml(xml_path):
    tree = ET.parse(xml_path)
    return [(elem.text, elem.attrib['type']) for elem in tree.iter() if elem.tag == 'premise']

# Retrieve premises from all XML files
for split in ['positive', 'negative']:
    for path in glob.glob(os.path.join(f'v2.0/{split}', '*.xml')):
        try:
            for premise, premise_type in parse_xml(path):
                df_premises = df_premises.append({
                    'file_idx': path.split('/')[2].split('.')[0],
                    'file_path': path,
                    'split': path.split('/')[1],
                    'premise': premise,
                    'type': premise_type
                }, ignore_index=True)
        except ET.ParseError:
            pass

/var/folders/5g/c4j2vy1n4_j9z32rb3hg6q740000gn/T/ipykernel_49603/3307439083.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_premises = df_premises.append({
/var/folders/5g/c4j2vy1n4_j9z32rb3hg6q740000gn/T/ipykernel_49603/3307439083.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_premises = df_premises.append({
/var/folders/5g/c4j2vy1n4_j9z32rb3hg6q740000gn/T/ipykernel_49603/3307439083.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_premises = df_premises.append({
/var/folders/5g/c4j2vy1n4_j9z32rb3hg6q740000gn/T/ipykernel_49603/3307439083.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_premises = df_premis

In [53]:
df_premises

,file_idx,file_path,split,premise,type
0,88,v2.0/positive/88.xml,positive,[47% of all jobs are at risk of being automate...,logos
1,88,v2.0/positive/88.xml,positive,This number will grow grow until the vast majo...,logos_pathos
2,88,v2.0/positive/88.xml,positive,Since some of this automation will inevitably ...,logos_pathos
3,88,v2.0/positive/88.xml,positive,By fortifying themselves in their gated commun...,logos_pathos
4,88,v2.0/positive/88.xml,positive,"Once everyone is dead, they can simply be wipe...",logos_pathos
...,...,...,...,...,...
1953,85,v2.0/negative/85.xml,negative,the some few people that still know how to hun...,logos_pathos
1954,85,v2.0/negative/85.xml,negative,Machines don't have money either so they won't...,logos_pathos
1955,85,v2.0/negative/85.xml,negative,Cryptocurrency may help a little because it wo...,logos
1956,85,v2.0/negative/85.xml,negative,once they can administrate their own systems w...,logos
